<a href="https://colab.research.google.com/github/gkckdals3/ML_/blob/main/%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C(%EC%A0%84%EC%B2%B4%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%ED%95%99%EC%8A%B5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import numpy as np
import cv2

In [38]:
import glob
from google.colab import drive
import pandas as pd
drive.mount('/gdrive', force_remount=True)
#train 타겟값 target에 저장
target_table=pd.read_csv('/gdrive/MyDrive/SyntekaBio/train_output.csv').to_numpy()
target=target_table[:,1:].astype(float)
train_data =(sorted(glob.glob('/gdrive/MyDrive/SyntekaBio/train/PNG/*.png')))

Mounted at /gdrive


In [39]:
#전체 데이터 불러오기/데이터 10분에 1로 축소
for i in range (len(train_data)):
  train_data[i]=cv2.imread(train_data[i])
  train_data[i]=cv2.resize(train_data[i],dsize=(128, 128),interpolation=cv2.INTER_AREA)
  train_data[i]=train_data[i].astype(np.float32)/255
train_scaled=np.array(train_data)

In [40]:
from tensorflow import keras
#전체 이미지 데이터 학습
print('학습 데이터 shape : ', train_scaled.shape)
print('검증 데이터 shape : ', target.shape)


학습 데이터 shape :  (501, 128, 128, 3)
검증 데이터 shape :  (501, 1)


In [41]:
import keras
import tensorflow
#모델링 설계
#seed값 고정
tensorflow.random.set_seed(2)
model = keras.Sequential()
model.add(keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu', kernel_initializer='he_uniform', input_shape=(128,128,3)))
model.add(keras.layers.MaxPooling2D(padding='SAME'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [42]:
#모델 summary 확인
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 126, 126, 128)     3584      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 63, 63, 128)      0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 63, 63, 128)       0         
                                                                 
 flatten_4 (Flatten)         (None, 508032)            0         
                                                                 
 dense_12 (Dense)            (None, 256)               130056448 
                                                                 
 dropout_9 (Dropout)         (None, 256)               0         
                                                      

In [43]:
import tensorflow
from tensorflow.keras.optimizers import Adam
#모델 컴파일
model.compile(loss='binary_crossentropy',
              optimizer = Adam(learning_rate=0.01), 
              metrics=['accuracy'])


In [44]:
#모델 학습
history = model.fit(
            train_scaled, target, 
            batch_size = 32,
            epochs = 28

           
                    )

Epoch 1/28
16/16 [==============================] - 1s 34ms/step - loss: 640.7506 - accuracy: 0.5629
Epoch 2/28
16/16 [==============================] - 1s 32ms/step - loss: 22.4944 - accuracy: 0.5369
Epoch 3/28
16/16 [==============================] - 1s 33ms/step - loss: 1.6787 - accuracy: 0.5968
Epoch 4/28
16/16 [==============================] - 1s 32ms/step - loss: 0.5842 - accuracy: 0.7505
Epoch 5/28
16/16 [==============================] - 1s 32ms/step - loss: 0.3193 - accuracy: 0.8802
Epoch 6/28
16/16 [==============================] - 1s 32ms/step - loss: 0.2480 - accuracy: 0.9062
Epoch 7/28
16/16 [==============================] - 1s 32ms/step - loss: 0.2276 - accuracy: 0.9321
Epoch 8/28
16/16 [==============================] - 1s 32ms/step - loss: 0.2141 - accuracy: 0.9301
Epoch 9/28
16/16 [==============================] - 1s 32ms/step - loss: 0.1967 - accuracy: 0.9261
Epoch 10/28
16/16 [==============================] - 1s 32ms/step - loss: 0.1826 - accuracy: 0.9242
Epoch 

In [45]:
#테스트_data 불러오기
test_target_table=pd.read_csv('/gdrive/MyDrive/SyntekaBio/test_output_sample.csv').to_numpy()
test_target=target_table[:,1:].astype(float)
test_data =(sorted(glob.glob('/gdrive/MyDrive/SyntekaBio/test/PNG/*.png')))

In [46]:
#테스트 게이터 사이즈 축소 및 전처리
for i in range (len(test_data)):
  test_data[i]=cv2.imread(test_data[i])
  test_data[i]=cv2.resize(test_data[i],dsize=(128, 128),interpolation=cv2.INTER_AREA).astype(np.float32)/255
test_scaled=np.array(test_data)

In [47]:
#전처리된 데이터 구조 확인
test_scaled.shape

(126, 128, 128, 3)

In [48]:
모델예측 및 확인
pred=(model.predict(test_scaled))
pred[1:3]

array([[0.9963218 ],
       [0.00483535]], dtype=float32)

In [49]:
#값을 0또는 1로 표시하도록 함수선언
def binary_classification(self):
  for i in range (len(self)):
    if pred[i]<0.5:
      pred[i]=int(0)
    else:
      pred[i]=int(1)

In [53]:
# 선언한 함수를 사용해서 예측
binary_classification(pred)

In [51]:
#csv파일에 target값 입력
test_target=pd.read_csv('/gdrive/MyDrive/SyntekaBio/test_output_sample.csv')
test_target.iloc[:,1]=pred
test_target

,Sample ID,Control:0/Case:1
0,AZ-004,0.0
1,AZ-005,1.0
2,AZ-018,0.0
3,AZ-020,1.0
4,AZ-021,0.0
...,...,...
121,AZ-779,0.0
122,AZ-783,1.0
123,AZ-795,0.0
124,AZ-800,1.0


In [52]:
#csv파일로 해당 경로에 저장
test_target.to_csv('/gdrive/MyDrive/SyntekaBio/test_target.csv',index=False)